In [13]:
import sys
import os
sys.path.append("C:/Users/pablo/Documents/GitHub/")

from dotenv import load_dotenv
load_dotenv("C:/Users/pablo/Documents/GitHub/sid_ml_analysis/Object Detection/yolo_custom_detection")

from sid_modules.helperFunctions import BigQueryHelper, ConversionHelper #, OSMQueryHelper
bqh = BigQueryHelper()
cnv = ConversionHelper()
#osh = OSMQueryHelper()

In [63]:
df_images = bqh.loadImages()
df_images.head()

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 12.69rows/s]


,img_name,name,input_lat,input_lon,input_size,grid_size,zoom,lat_size,lon_size,lat_0,lon_0,m_per_px_lat,m_per_px_lon,deg_per_m_lat,deg_per_m_lon,deg_per_px_lat,deg_per_px_lon
0,Monnon_5_19_500.png,Monnon,9.842513,3.178864,500,5,19,2285,2500,9.845532,3.175517,0.294187,0.298123,0.000008,0.000009,0.000002,0.000003
1,Sansi Gando_5_19_500.png,Sansi Gando,9.947919,3.418565,500,5,19,2285,2500,9.950937,3.415034,0.294093,0.298051,0.000008,0.000009,0.000002,0.000003
2,Besen Gourou_5_19_500.png,Besen Gourou,9.905893,3.350370,500,5,19,2285,2500,9.908911,3.346920,0.294131,0.298072,0.000008,0.000009,0.000002,0.000003
3,Barkedje_5_19_500.png,Barkedje,10.025566,3.287887,500,5,19,2285,2500,10.028584,3.284179,0.294023,0.298091,0.000008,0.000010,0.000002,0.000003
4,Boudal_5_19_500.png,Boudal,9.866730,3.172789,500,5,19,2285,2500,9.869749,3.169404,0.294166,0.298124,0.000008,0.000009,0.000002,0.000003


In [64]:
names =[] 

for index, rows in df_images.iterrows():
    names.append(rows.img_name) 
print(names) 

['Monnon_5_19_500.png', 'Sansi Gando_5_19_500.png', 'Besen Gourou_5_19_500.png', 'Barkedje_5_19_500.png', 'Boudal_5_19_500.png']


In [65]:
cnv.point_px_to_deg(0,0, df_images[0:1])

(9.84553185161675, 3.17551659337868)

In [66]:
# Pillar fila a fila los nombres de las imágenes. Cambiar el bucle

### Aquí hacemos detección y lo sacamos en pixeles

In [83]:
import os
import cv2
import numpy as np
import glob
import random
import pandas as pd

from math import radians, cos, sin, asin, sqrt, atan

# Load Yolo
net = cv2.dnn.readNet("yolov3_training_last.weights", "yolov3_testing.cfg")

# Name custom object
classes = ["buildings"]
list_nobj = []
path = r"C:\\Users\\pablo\\Documents\\GitHub\\sid_ml_analysis\\Object Detection\\yolo_custom_detection\\predictions\\"

# Images path
#  --> Cambiar para que acceda directamente a las fotos del drive / bucket de GCP de Alfon...
#images_path = glob.glob(r"C:\Users\pablo\Documents\GitHub\sid_ml_analysis\Object Detection\yolo_custom_detection\predictions\*.png")

layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 1))

#Dataframe donde vamos a guardar los datos de las casas detectadas:
df = pd.DataFrame(columns=['district','village','analysis_method','x', 'y', 'center_x', 'center_y', 'lat', 'lon', 'perim',
                          'area', 'boundary_lat', 'boundary_lon', 'point'])
                  
#i = 0 #índice del df_images para calcular conversiones de cada una de las imágenes
# loop through all the images
for name in names:
    img_path = path + name
    #saco el nombre del fichero
    base = os.path.basename(img_path)
    #tener en cuenta que separa el nombre por "_" para sacar sólo el nombre del pueblo
    name = name.split("_")[0]
    print(name)
    
    # Loading image
    img = cv2.imread(img_path)
    #Reducimos un poco la escala para ver la imagen entera por pantalla (parámetros fx fy)
    img = cv2.resize(img, None, fx=1, fy=1)
    height, width, channels = img.shape #Estas medidas están en pixeles

    # Detecting objects
    blob = cv2.dnn.blobFromImage(img, 1/255, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []
    
    for out in outs:
        n_objetos = 0
        
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            
            if confidence > 0.3:
                # Object detected
                n_objetos += 1
                
                # Centros
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
                         
                ######################################################
                lat_pto = cnv.point_px_to_deg(center_y,center_x, df_images[index:index+1])[0]
                lon_pto = cnv.point_px_to_deg(center_y,center_x, df_images[index:index+1])[1]
                ######################################################
                
                #Añado los objetos detectados al df
                df2 = pd.DataFrame([['Nikki',name,'ML_analysis',x, y, center_x, center_y,
                                     lat_pto, lon_pto, None, None, None, None, None]],
                                   columns=['district','village','analysis_method','x', 'y',
                                            'center_x', 'center_y', 'lat', 'lon', 'perim',
                                            'area', 'boundary_lat', 'boundary_lon', 'point'])
                
                df2["village"] = name
                df= df.append(df2)
                
    list_nobj.append(n_objetos)    
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    #font = cv2.FONT_HERSHEY_PLAIN
    
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            #label = str(classes[class_ids[i]])
            label = ""
            color = (255, 0, 0) 
            
            #cv2.rectangle(imagen, start point, end point, color, grosor de la línea)
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 1)
            
            #Si quisieramos escribir el nombre de la clase detectada:
            #cv2.putText(img, label, (x, y + 30), font, 3, color, 2)
    
    cv2.imshow("Image", img)
    key = cv2.waitKey(0)
    #i += 1

print(f'Número de casas por imagen: {list_nobj}')
cv2.destroyAllWindows()
df = df.reset_index()

df.drop(['index','x', 'y', 'center_x', 'center_y'], axis = 1, inplace = True)


Monnon
Sansi Gando
Besen Gourou
Barkedje
Boudal
Número de casas por imagen: [62, 7, 8, 2, 1]


In [84]:
df.head()

,district,village,analysis_method,lat,lon,perim,area,boundary_lat,boundary_lon,point
0,Nikki,Monnon,ML_analysis,9.869404,3.172177,None,None,None,None,None
1,Nikki,Monnon,ML_analysis,9.869046,3.171863,None,None,None,None,None
2,Nikki,Monnon,ML_analysis,9.869051,3.171882,None,None,None,None,None
3,Nikki,Monnon,ML_analysis,9.868339,3.171351,None,None,None,None,None
4,Nikki,Monnon,ML_analysis,9.868327,3.171367,None,None,None,None,None


### Descargo los building que están ya en BQ

In [82]:
df_buildings = bqh.loadSIDBuildings()
df_buildings.head()

Downloading: 100%|████████████████████████████████████████████████████████████████| 251/251 [00:00<00:00, 801.31rows/s]


,district,village,analysis_method,lat,lon,perim,area,boundary_lat,boundary_lon,point
0,Nikki,Monnon,color_analysis,9.846768,3.173095,None,None,None,None,POINT (3.17309545956545 9.84676801050273)
1,Nikki,Monnon,color_analysis,9.844994,3.175996,None,None,None,None,POINT (3.17599624243513 9.844994061474161)
2,Nikki,Monnon,color_analysis,9.844457,3.173685,None,None,None,None,POINT (3.1736846704566 9.844456539625311)
3,Nikki,Monnon,color_analysis,9.846947,3.173741,None,None,None,None,POINT (3.17374119842393 9.84694735076029)
4,Nikki,Monnon,color_analysis,9.845922,3.176059,None,None,None,None,POINT (3.17605882568637 9.845921610603931)


In [85]:
df_buildings.shape

(251, 10)

### Unimos los Dataframe de BQ y las predicciones hechas

In [96]:
df_final = pd.concat([df_buildings, df], ignore_index = True)
df_final["polygon"] = ""
df_final.shape

(331, 11)

In [97]:
df_final.head()

,district,village,analysis_method,lat,lon,perim,area,boundary_lat,boundary_lon,point,polygon
0,Nikki,Monnon,color_analysis,9.846768,3.173095,None,None,None,None,POINT (3.17309545956545 9.84676801050273),
1,Nikki,Monnon,color_analysis,9.844994,3.175996,None,None,None,None,POINT (3.17599624243513 9.844994061474161),
2,Nikki,Monnon,color_analysis,9.844457,3.173685,None,None,None,None,POINT (3.1736846704566 9.844456539625311),
3,Nikki,Monnon,color_analysis,9.846947,3.173741,None,None,None,None,POINT (3.17374119842393 9.84694735076029),
4,Nikki,Monnon,color_analysis,9.845922,3.176059,None,None,None,None,POINT (3.17605882568637 9.845921610603931),


In [98]:
df_final.tail()

,district,village,analysis_method,lat,lon,perim,area,boundary_lat,boundary_lon,point,polygon
326,Nikki,Besen Gourou,ML_analysis,9.865143,3.172044,None,None,None,None,None,
327,Nikki,Besen Gourou,ML_analysis,9.865172,3.172247,None,None,None,None,None,
328,Nikki,Barkedje,ML_analysis,9.867759,3.172805,None,None,None,None,None,
329,Nikki,Barkedje,ML_analysis,9.866288,3.171692,None,None,None,None,None,
330,Nikki,Boudal,ML_analysis,9.864940,3.172748,None,None,None,None,None,


### Cargamos a BigQuery

In [99]:
bqh.saveSIDBuildings(df_final)

1it [00:02,  2.01s/it]
